<a href="https://colab.research.google.com/github/AliKarimiENT/Machine_Translation_EN_FA/blob/main/Machine_Translation_EN_FA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Translation Project

# Dataset




> The data are located in Google Drive. The TEP.en-fa.en file contains English sentences with their Farsi translation in the TEP.en-fa.fa file. 

Load the English and Farsi data from these files from running the cell below.